In [8]:
# In a Jupyter Notebook cell
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:

import numpy as np
print(np.__version__)
from mirage import WordCountingChunkingAlgorithm, FolderRawStorage, WhooshChunkStorage
from mirage import HuggingFaceEmbedder, TextNormalizer
from mirage import NatashaSentenceChunking, SemanticChunking
import os
import pickle

1.26.4


In [11]:
embedder = HuggingFaceEmbedder(model_name='BAAI/bge-m3', normalizer=True)

In [12]:
import logging
logging.basicConfig(level=logging.ERROR)


documents = FolderRawStorage('data_test')
 # тут заменить на нужную папку
chunks = WhooshChunkStorage(scoring_function="BM25F", normalizer=True)
# algorithm = WordCountingChunkingAlgorithm(documents, chunks, words_amount=100)


In [13]:

algorithm = SemanticChunking(raw_storage=documents, chunk_storage=chunks, embedder=embedder, threshold=0.5, max_chunk_size=1000)
algorithm.execute(visualize=True)

Splitting text into the sentences


611it [00:00, 50276.02it/s]


creating embeddings of the sentences for semantic grouping


100%|██████████| 611/611 [00:58<00:00, 10.53it/s]


Adding chunks to the storage


100%|██████████| 427/427 [00:02<00:00, 150.76it/s]


1

In [14]:
import random
index = chunks.get_indexes()[random.randint(1, 50)]
print(len(chunks[index]))
chunks[index]

88


'Этот маленький ребус он послал нам с первой почтой, а сам выехал сюда ближайшим поездом.'

In [15]:
from mirage.index.vector_index.ram_vector_index import L2RAMVectorIndex
from mirage import FaissIndexFlatL2, FaissIndexIVFPQR
vector_index = FaissIndexIVFPQR(dimensionality=embedder.get_dimensionality())
embedder.convert_chunks_to_vector_index(chunks, vector_index, visualize=True)

Converting ChunkStorage to VectorIndex


100%|██████████| 427/427 [00:43<00:00,  9.85it/s]


In [17]:
query = 'убитая женщина'
query_vector = embedder.embed(text=query)
# print(query_vector)
answer = vector_index.query(
    query_vector, top_k=20
)
print([element.score for element in answer])
[chunks[element.chunk_storage_key] for element in answer]

[0.8335791230201721, 0.8566122055053711, 0.8726161122322083, 0.8734219670295715, 0.8924689292907715, 0.8948429822921753, 0.929951012134552, 0.9746432304382324, 0.9803217649459839, 0.9977196455001831, 1.003746509552002, 1.0152220726013184, 1.01560640335083, 1.0271902084350586, 1.0438331365585327, 1.0458725690841675, 1.0471690893173218, 1.0567314624786377, 1.0579125881195068, 1.0834579467773438]


['— Она была найдена тяжело раненной возле своего мертвого мужа.',
 'Так рассказывают служанки. Он умер, она при смерти.',
 'Вы кончили тем, что убили этого благородного человека и довели его жену до самоубийства.',
 'Возле окна корчилась его жена, прислонясь головой к стене. Рана ее была ужасна — кровь залила половину лица.',
 'Их хозяин лежал посреди комнаты лицом вниз. Он был мертв.',
 'Моя жена получила письмо из Америки — на конверте была американская марка. Жена смертельно побледнела, прочла письмо и швырнула в огонь.',
 '— Если Илей умрет, мне все равно, что будет со мною, — сказал американец.',
 'Доктор удалился к своим пациентам.',
 'Я увидел, как из-за угла выползла темная согнутая фигура и уселась перед дверью. Схватив револьвер, я рванулся вперед, но жена судорожно обняла меня и удержала на месте. Я пытался оттолкнуть ее, но она вцепилась в меня еще отчаяннее.',
 'Он умер мгновенно и безболезненно.',
 '— Вы трогали убитого? — спросил Холме.',
 '— Его несомненно закрыла женщ

In [18]:
chunks.query('убийство')

7.189961190126932


[QueryResult(score=7.189961190126932, chunk_storage_key='76776b4e-438f-4fc5-98f8-e754a1ac88d4', vector=None),
 QueryResult(score=4.863961050920773, chunk_storage_key='d6fc642c-4f88-4e29-8b4c-4ea1e506c992', vector=None)]

In [23]:
from mirage.inference.quorums.RusVectoresQuorum import RusVectoresQuorum


quorum = RusVectoresQuorum(
    chunk_storage=chunks, 
    path_to_model='mirage\inference\quorums\model.bin',
    POS_thresholds={"ADJ": 0.5, "VERB": 0.65},
    visualize=True,
    max_combinations=25)
quorum.query("кто убил эту красивую и привлекательную женщину?")

Amount of vectors loaded in quorum: 189193
word "быть" is presented in word vectors: True
set of POS in w2v model: {'PROPN', 'X', 'NOUN', 'VERB', 'ADV', 'INTJ', 'ADJ', 'SYM', 'NUM'}
Rules of synonimization: {'ADJ': 0.5, 'VERB': 0.65}
 For word "убил" obtained synonims:
[('убивать_VERB', 0.7964877486228943), ('застреливать_VERB', 0.704032838344574), ('убить_ADJ', 0.6715459227561951), ('застрелить_VERB', 0.6328871250152588), ('убивать_NOUN', 0.6270444989204407), ('убитый_ADJ', 0.6136345863342285), ('убивать_ADJ', 0.5941329598426819), ('погибать_VERB', 0.5825400352478027), ('казнять_VERB', 0.573449432849884), ('зарезать_VERB', 0.5693537592887878), ('зарублять_VERB', 0.5649162530899048), ('расстреливать_VERB', 0.5634232759475708), ('убитый_VERB', 0.5592145919799805), ('ранить_VERB', 0.558910608291626), ('зарубить_VERB', 0.5410849452018738), ('избить_VERB', 0.531154990196228), ('застрелиться_VERB', 0.5310904383659363), ('изувечить_VERB', 0.5287972092628479), ('изувечивать_VERB', 0.525049030

[QueryResult(score=4.235756319691941, chunk_storage_key='18cb715c-dd8f-45a2-93a0-b40cae5ec09f', vector=None),
 QueryResult(score=6.308011912347461, chunk_storage_key='b3cfce21-e1d5-4520-8533-b6344f049728', vector=None),
 QueryResult(score=6.006355784088442, chunk_storage_key='215ee618-aff7-4829-83d9-cc2b831a5f72', vector=None),
 QueryResult(score=4.06325984511512, chunk_storage_key='dcd5cf3c-8e34-4187-b5e8-fcad76fcd387', vector=None),
 QueryResult(score=5.0419157615989665, chunk_storage_key='570e6cda-8b72-4bb1-b306-bb81b62a4cc4', vector=None),
 QueryResult(score=5.194963006640833, chunk_storage_key='5951ef07-7dda-4768-beae-a27d9efb077e', vector=None),
 QueryResult(score=6.8618212833083625, chunk_storage_key='4330f942-8b47-45be-b307-43ee20669153', vector=None),
 QueryResult(score=2.467047428704616, chunk_storage_key='586b85b3-e6d1-45d1-8e6b-56b9cb927672', vector=None),
 QueryResult(score=6.8618212833083625, chunk_storage_key='9c27a6d0-ce9c-4aca-a3a0-71d5dfb97aac', vector=None),
 QueryRe

In [ ]:
from mirage.inference.prompters.APILLM import LLM
llm = LLM()
llm.do_request(query=query,
               chunk_storage=chunks,
               indexes=[element.chunk_storage_key for element in answer],
               prompt='')